本项目用于模拟，N个请求，发送给1个或多个线程实例时，其总的耗时，结果如下：

|序号|任务情况|1个请求|2个请求|10个请求|
|---|---|---|---|---|
|A|1种任务的1个实例|619|1311|6640|
|B|1种任务的2个实例|446|1134|5925|
|C|2种任务各1个实例|1036|2565|11197|
|D|2种任务的2个实例|918|2100|10860|

结论：
- 看A、单纯增加请求数量，其耗时几乎成倍升高
- 对比A、B，C、D，增加线程实例，耗时减少，但不是和增加线程数量同倍减少
- 对比A、B，C、D，使用多线程处理多个请求确实能带来性能提升

In [1]:
#include <iostream>
#include <thread>
using namespace std;

In [2]:
// MyTask假设是待实例化的线程函数，它具有两个功能
// initStatue：记录线程状态customStatue、假设任务时间customSleepTime
// showStatue：输出线程开始时刻、假设执行任务、输出线程结束时刻

class MyTask
{
    public:
        MyTask(){}
        ~MyTask(){}

        // 初始化类变量
        int initStatue(int customStatue, int customSleepTime){
            seStatue = customStatue;
            setSleepTime = customSleepTime;
            return 0;
        }

        // 待调用的函数
        void showStatue(int customStatue){
        std::cout<< customStatue << "," << seStatue <<" start call:"<< clock() << endl;
        // Sleep(setSleepTime); // windows写法
        std::this_thread::sleep_for(std::chrono::milliseconds(setSleepTime));
        //std::cout << customStatue <<","<<setFunctionStatue << std::endl;
        //std::cout << customStatue<<","<< seStatue <<" From Thread ID : " << std::this_thread::get_id() << "\n";
        std::cout<<customStatue << "," << seStatue <<" end call:"<< clock() << endl;
        }
    private:
        // 每个任务的状态
        int seStatue;
        // 每个任务休眠的时间
        int setSleepTime;
}

In [3]:
clock_t startTime, endTime;
double cost;
std::vector<int> threadEnum{1,2,10}; // 自定义请求次数

In [4]:
// 初始化1个线程实例
std::map<int, MyTask> mapMyTask1;
MyTask myTask1;
myTask1.initStatue(0,100);
mapMyTask1[0] = myTask1;

// 分别使用{1,2,10}次请求1种任务的1个实例
for (int threadNum : threadEnum) {
    startTime = clock();//计时开始
    std::vector<std::thread> mutilThread;
    for (int ind = 0; ind < threadNum; ind++) {
        std::thread myThread(&MyTask::showStatue, mapMyTask1[0], ind);
        mutilThread.push_back(std::move(myThread));
    }
    for (std::thread& th : mutilThread)
    {
        if (th.joinable())
            th.join();
    }
    endTime = clock();//计时结束
    cost = (double)(endTime - startTime);
    std::cout << threadNum << "->" << endTime << "-" << startTime << ":" << cost << std::endl;
}

0,0 start call:1291197
0,0 end call:1291502
1->1291714-1291095:619
0,0 start call:1292029
1,0 start call:1292307
0,0 end call:1292746
1,0 end call:1293045
2->1293211-1291900:1311
0,0 start call:1293483
3,0 start call:1293989
9,08 start call:,0 start call:1294758
6,0 start call:1295264
1294860
4,0 start call:1295738
5,0 start call:1296042
2,0 start call:1296338
7,0 start call:1296638
1,0 start call:1296918
0,0 end call:1297268
3,0 end call:1297347
9,0 end call:1297602
6,0 end call:1298151
8,0 end call:1298412
4,0 end call:1298670
5,0 end call:1298876
2,0 end call:1299185
7,0 end call:1299396
1,0 end call:1299705
10->1300053-1293413:6640


In [5]:
// 1种任务，初始化2个
std::map<int, MyTask> mapMyTask2;
MyTask myTask2;
myTask2.initStatue(2, 100);
mapMyTask2[0] = myTask2;
MyTask myTask3;
myTask3.initStatue(3, 100);
mapMyTask2[1] = myTask3;

// 分别使用{1,2,10}请求1种任务的2个实例
for (int threadNum : threadEnum) {
    startTime = clock();//计时开始
    std::vector<thread> mutilThread;
    for (int ind = 0; ind < threadNum; ind++) {
        thread myThread(&MyTask::showStatue, mapMyTask2[ind%2], ind);
        mutilThread.push_back(std::move(myThread));
    }
    for (std::thread& th : mutilThread)
    {
        if (th.joinable())
            th.join();
    }
    endTime = clock();//计时结束
    cost = (double)(endTime - startTime);
    std::cout << threadNum << "->" << endTime << "-" << startTime << ":" << cost << std::endl;
}

0,2 start call:1388607
0,2 end call:1388822
1->1388985-1388539:446
0,2 start call:1389346
1,3 start call:1389593
0,2 end call:1389897
1,3 end call:1389954
2->1390356-1389222:1134
0,2 start call:1390692
4,2 start call:1391121
29,,3 start call:2 start call:1391620
5,3 start call:1391855
6,2 start call:1392120
8,2 start call:1392412
3,3 start call:1392644
7,3 start call:1392917
1,3 start call:1393155
1391620
0,2 end call:1393689
4,2 end call:1393722
9,3 end call:1394456
5,3 end call:1394499
6,2 end call:1394716
8,2 end call:1395239
3,3 end call:1395463
7,3 end call:1395683
1,3 end call:1395877
2,2 end call:1396090
10->1396497-1390572:5925


In [6]:
// 2种任务，每个初始化1个
std::map<int, MyTask> mapMyTask3;
MyTask myTask4;
myTask4.initStatue(4, 100);
mapMyTask3[0] = myTask4;
MyTask myTask5;
myTask5.initStatue(5, 100);
mapMyTask3[1] = myTask5;

// 分别使用{1,2,10}请求2种任务的各1个实例
for (int threadNum : threadEnum) {
    startTime = clock();//计时开始
    std::vector<thread> mutilThread1;
    std::vector<thread> mutilThread2;
    for (int ind = 0; ind < threadNum; ind++) {
        thread myThread1(&MyTask::showStatue, mapMyTask3[0], ind);
        thread myThread2(&MyTask::showStatue, mapMyTask3[1], ind);
        mutilThread1.push_back(std::move(myThread1));
        mutilThread2.push_back(std::move(myThread2));
    }
    for (std::thread& th : mutilThread1)
    {
        if (th.joinable())
            th.join();
    }
    for (std::thread& th : mutilThread2)
    {
        if (th.joinable())
            th.join();
    }

    endTime = clock();//计时结束
    cost = (double)(endTime - startTime);
    std::cout << threadNum << "->" << endTime << "-" << startTime << ":" << cost << std::endl;
}

0,4 start call:1485972
0,5 start call:1486120
0,4 end call:1486430
0,5 end call:1486600
1->1486925-1485889:1036
0,4 start call:1487221
0,5 start call:1487560
1,5 start call:1487842
1,4 start call:1488152
0,4 end call:1488391
0,5 end call:1488593
1,5 end call:1488661
1,4 end call:1488756
2->1489661-1487096:2565
0,4 start call:1489881
0,5 start call:1490284
15,,5 start call:1490927
7,5 start call:1491378
3,4 start call:1491814
4,5 start call:1492169
5,5 start call:1492393
2,4 start call:1492694
6,4 start call:1492944
7,4 start call:1493129
8,4 start call:1493286
9,4 start call:1493513
4,4 start call:1493756
2,5 start call:1494073
6,5 start call:1494342
8,5 start call:1494615
4 start call:1494853
3,5 start call:91495146
,5 start call:1495389
1,4 start call:1495618
0,4 end call:1495822
0,5 end call:1495831
1,5 end call:1495945
7,5 end call:1496625
3,4 end call:1497124
4,5 end call:1497139
5,5 end call:1497431
2,4 end call:1497858
6,4 end call:1498069
7,4 end call:1498259
8,4 end call:14985

In [7]:
// 2种任务，每个初始化2个
std::map<int, MyTask> mapMyTask4;
MyTask myTask6;
myTask6.initStatue(6, 100);
mapMyTask4[0] = myTask6;
MyTask myTask7;
myTask7.initStatue(7, 100);
mapMyTask4[1] = myTask7;
MyTask myTask8;
myTask8.initStatue(8, 100);
mapMyTask4[0] = myTask8;
MyTask myTask9;
myTask9.initStatue(9, 100);
mapMyTask4[1] = myTask9;

// 分别使用{1,2,10}请求2种任务的各2个实例
for (int threadNum : threadEnum) {
    startTime = clock();//计时开始
    std::vector<thread> mutilThread1;
    std::vector<thread> mutilThread2;
    for (int ind = 0; ind < threadNum; ind++) {
        thread myThread1(&MyTask::showStatue, mapMyTask4[ind%2], ind);
        thread myThread2(&MyTask::showStatue, mapMyTask4[ind%2], ind);
        mutilThread1.push_back(std::move(myThread1));
        mutilThread2.push_back(std::move(myThread2));
    }
    for (std::thread& th : mutilThread1)
    {
        if (th.joinable())
            th.join();
    }
    for (std::thread& th : mutilThread2)
    {
        if (th.joinable())
            th.join();
    }
    endTime = clock();//计时结束
    cost = (double)(endTime - startTime);
    std::cout << threadNum << "->" << endTime << "-" << startTime << ":" << cost << std::endl;
}

0,8 start call:1596002
0,8 start call:1596157
0,8 end call:1596414
0,8 end call:1596577
1->1596847-1595929:918
0,8 start call:1597174
0,8 start call:1597428
1,9 start call:1597638
1,9 start call:1597833
0,8 end call:1598077
0,8 end call:1598103
1,9 end call:1598409
1,9 end call:1598689
2->1599100-1597000:2100
0,8 start call:1599442
2,8 start call:1599791
5,9 start call:11600264
7,9 start call:1600720
29,9 start call:1601115
4,8 start call:1601413
,9 start call:1,1601600
6,8 start call:1601825
7,9 start call:1602032
8,8 start call:1602192
3,9 start call:1602464
9,9 start call:1602661
4,8 start call:91603007 start call:
6,8 start call:1603284
0,8 start call:1603472
3,9 start call:16030181603743
8,8 start call:
1604001
,8 start call:5,9 start call:1604441
1604284
0,8 end call:1604863
2,8 end call:1604893
5,9 end call:1604974
7,9 end call:1605838
9,9 end call:1606098
4,8 end call:1606412
1,9 end call:1606691
6,8 end call:1606997
7,9 end call:1607207
8,8 end call:1607467
3,9 end call:160768